# Preparação do ambiente

## Bibliotecas

In [2]:
import numpy as np
import os.path as osp
import pandas as pd

from joblib import load

## Constantes e funções auxiliares

In [3]:
FILE_MODEL = '../models/clf_marketplaces.joblib'
FILE_SUPERVISAO_MERCADO = '../datasets/inspecao_ecommerce/supervisao_mercado.xlsx'

In [4]:
class MarketPlacesPredictor():

    def __init__(self, file_model):
        if osp.exists(file_model):
            self.model = load(file_model)
        else:
            self.model = None
            print(f'Model file "{file_model}" not found')

    def predict(self, X):
        if self.model is not None:
            predicted_class = self.model.predict(X)
            predicted_proba = self.model.predict_proba(X)[:,1]
            return np.column_stack((predicted_class,predicted_proba))
        else:
            print('Model is not loaded')
            return None


# Carga e prepação dos dados

In [5]:
dict_df_mercado = pd.read_excel(
    FILE_SUPERVISAO_MERCADO,
    sheet_name=None,
    usecols=[2,8,10],
    names=['texto_busca', 'descricao', 'passivel_homologacao'],
    true_values=['Sim', 'sim'],
    false_values=['Não','não'],
    na_values=['-'])

df_mercado = pd.concat(dict_df_mercado.values())
df_mercado = df_mercado.dropna()
df_mercado['passivel_homologacao'] = df_mercado['passivel_homologacao'].astype(int)
df_mercado

,texto_busca,descricao,passivel_homologacao
0,Smartphone,Positivo Twist 4 Pro S518 Dual SIM 64 GB preto...,1
1,Smartphone,Xiaomi Pocophone Poco X6 Pro 5G Dual SIM 512 G...,1
2,Smartphone,"Smartphone Moto G22 Tela De 6,5 128 Gb 4 Ram P...",1
3,Smartphone,Smartphone Infinix Smart 6 64 Gb Dual Chip 2gb...,1
4,Smartphone,Smartphone Samsung Galaxy A21s Tela 6.5 64gb 4...,1
...,...,...,...
170,Bluetooth Wifi,fone bluetooth sem fio beatfade preto,1
171,Bluetooth Wifi,Fone Bluetooth Sem Fio Wireless Compatível Com...,1
172,Bluetooth Wifi,Fone Bluetooth Sem Fio Preto Caixinha Para Sam...,1
173,Bluetooth Wifi,Fone De Ouvido Bluetooth Sem Fio Tipo Ambie De...,1


# Análise

In [6]:
predictor = MarketPlacesPredictor(FILE_MODEL)
predictor.model

Pipeline(steps=[('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...])),
                ('transformer', TfidfTransformer()),
                ('clf', SGDClassifier(alpha=1e-05, loss='log_loss'))])

In [7]:
X = df_mercado.descricao.values
y = df_mercado.passivel_homologacao.values

prediction = predictor.predict(X)

if prediction is not None:
    predicted_class = prediction[:,0]
    predicted_proba = prediction[:,1]
    
    df_mercado['classe_predita'] = predicted_class.astype(int)
    df_mercado['probabilidade'] = predicted_proba
    
    print('Accuracy of model on validation set: {:.3f}'
         .format(predictor.model.score(X, y)))
    
    df_mercado.pivot_table(index='passivel_homologacao',columns='classe_predita',values='descricao',aggfunc='count')

Accuracy of model on validation set: 0.891


In [8]:
# mesmo que seja um item a ser preditor, o argumento deve ser uma lista
predictor.predict(['Repetidor De Sinal Internet 600mbps Branco 110v/220v Wifi'])

array([[1.       , 0.9919337]])

In [9]:
# para predizer a classe de vários itens, basta passar um iterável como parâmetro
to_predict = df_mercado.groupby('passivel_homologacao').sample(2,random_state=111)['descricao'].to_list()
predictor.predict(to_predict)

array([[0.        , 0.27229482],
       [0.        , 0.11959836],
       [1.        , 0.99560204],
       [1.        , 0.95066412]])

In [10]:
df_mercado.groupby('passivel_homologacao').sample(2,random_state=111)

,texto_busca,descricao,passivel_homologacao,classe_predita,probabilidade
65,Carregador para Smartphone,Cabo USB-C de transferência/carregador de foto...,0,0,0.272295
134,TV Box,Cama Box Baú Casal + Colchão Casal Molas Ensac...,0,0,0.119598
11,Smartphone,"Realme C53 RMX 3760 4G LTE. RAM 8GB+256GB, Dou...",1,1,0.995602
139,TV Box,Box Tv Vontar H618 4x64gb 5g Android 12 Origin...,1,1,0.950664
